In [1]:
from boto3 import client
import json
import pandas as pd
import dotenv
import os
import sys
import boto3
from io import BytesIO
import csv

In [2]:
# Read from the json the perform clean and write csv

In [3]:
# Insert the parent path relative to this notebook, so we can import from the src folder.
sys.path.insert(0, "..")

# Load Environment Variables
dotenv.load_dotenv("../.env")

region_name = os.getenv("AWS_REGION")
access_key = os.getenv("AWS_ACCESS_KEY_ID")
access_secret = os.getenv("AWS_SECRET_ACCESS_KEY")
bucket_name = os.getenv("BUCKET_NAME")

In [4]:
  # Create an S3 client and resource
s3 = boto3.client("s3")
s3 = boto3.resource(
        service_name="s3",
        region_name=region_name,
        aws_access_key_id=access_key,
        aws_secret_access_key=access_secret,
    )

In [5]:
# Converts JSON data to CSV format, assuming the JSON file contains a list of dictionaries with consistent keys, and writes the data into a CSV file.

In [6]:

def json_to_csv(json_file, csv_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    if "items" in data and isinstance(data["items"], list) and len(data["items"]) > 0:
        # Extract field names from the first item
        field_names = list(data["items"][0].keys())

        with open(csv_file, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=field_names)
            writer.writeheader()

            # Write each item as a row in the CSV file
            for item in data["items"]:
                writer.writerow(item)
    else:
        print("JSON data is not in the expected format.")



In [7]:
# Reads JSON data from the file "json_thames_water_alert_stream.json", 
# Converts it into CSV format, and writes the result into "thames_water_alert_stream.csv".

In [8]:

json_file = "json_thames_water_alert_stream.json"
csv_file = "thames_water_alert_stream.csv"
json_to_csv(json_file, csv_file)


In [9]:
# Reads a CSV file, 
# Converts the 'DateTime' column to datetime format, 
# Converts all string columns to lowercase, 
# Removes rows with missing values, and returns the cleaned DataFrame.

In [10]:
import pandas as pd

def clean_csv_dataset(csv_file):
    # Load CSV into a DataFrame
    df = pd.read_csv(csv_file)

    # Convert 'DateTime' column to datetime format
    df['DateTime'] = pd.to_datetime(df['DateTime'])

     # Convert all string columns to lowercase
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Drop any rows with missing values
    df.dropna(inplace=True)

    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df

In [11]:
# Clean the CSV file "thames_water_alert_stream.csv", 
# stores the cleaned data in the variable "cleaned_dte_alert_stream", and prints the first few rows of the cleaned DataFrame.

In [12]:
# Usage example:
csv_file = "thames_water_alert_stream.csv"
cleaned_dte_alert_stream = clean_csv_dataset(csv_file)
print(cleaned_dte_alert_stream.head())


  LocationName PermitNumber LocationGridRef       X       Y  \
0    clanfield    cssc.1001      sp29000181  429000  201810   
1     cottered    cssc.1038      tl31522968  531520  229680   
2      barkway    catm.3093      tl38783476  538780  234760   
3   willingale    cssc.1456      tl58950707  558950  207070   
4   willingale    cssc.1456      tl58950707  558950  207070   

        ReceivingWaterCourse AlertType            DateTime  
0      marsh lane west ditch     start 2024-05-06 07:00:00  
1             cottered brook      stop 2024-05-06 06:30:00  
2                 river quin      stop 2024-05-06 06:15:00  
3  tributary of river roding     start 2024-05-06 06:00:00  
4  tributary of river roding      stop 2024-05-06 05:30:00  


In [13]:
cleaned_dte_alert_stream.head()

,LocationName,PermitNumber,LocationGridRef,X,Y,ReceivingWaterCourse,AlertType,DateTime
0,clanfield,cssc.1001,sp29000181,429000,201810,marsh lane west ditch,start,2024-05-06 07:00:00
1,cottered,cssc.1038,tl31522968,531520,229680,cottered brook,stop,2024-05-06 06:30:00
2,barkway,catm.3093,tl38783476,538780,234760,river quin,stop,2024-05-06 06:15:00
3,willingale,cssc.1456,tl58950707,558950,207070,tributary of river roding,start,2024-05-06 06:00:00
4,willingale,cssc.1456,tl58950707,558950,207070,tributary of river roding,stop,2024-05-06 05:30:00


In [14]:
# Uploads a local file to an Amazon S3 bucket specified by the parameters provided and prints a confirmation message.

In [15]:
def upload_file_to_s3(local_file, bucket_name, key_name, region_name, access_key, access_secret):
    # Create an S3 client
    s3 = boto3.client(
        service_name='s3',
        region_name=region_name,
        aws_access_key_id=access_key,
        aws_secret_access_key=access_secret
    )

    # Upload file to S3 bucket
    s3.upload_file(local_file, bucket_name, key_name)
    print(f"File uploaded to S3 bucket '{bucket_name}' with key '{key_name}'.")


In [16]:
# Saves the DataFrame "cleaned_dte_alert_stream" to a local CSV file, 
# Then uploads it to an Amazon S3 bucket, specifying the bucket name, key name, region, access key, and access secret.

In [17]:

local_file = "cleaned_dte_alert_stream.csv"
bucket_name = bucket_name
key_name = "Thames_water_data_set/processed_csv_data/thames_water_alert_stream.csv"  # Replace with the desired key name in your S3 bucket
region_name = region_name  # Replace with the AWS region of your S3 bucket
access_key = access_key  # Replace with your AWS access key
access_secret = access_secret  # Replace with your AWS access secret

cleaned_dte_alert_stream.to_csv('cleaned_dte_alert_stream.csv', index=False)


upload_file_to_s3(local_file, bucket_name, key_name, region_name, access_key, access_secret)



File uploaded to S3 bucket '2307-03-hydraulic-overload' with key 'Thames_water_data_set/processed_csv_data/thames_water_alert_stream.csv'.


In [18]:
cleaned_dte_alert_stream.head()

,LocationName,PermitNumber,LocationGridRef,X,Y,ReceivingWaterCourse,AlertType,DateTime
0,clanfield,cssc.1001,sp29000181,429000,201810,marsh lane west ditch,start,2024-05-06 07:00:00
1,cottered,cssc.1038,tl31522968,531520,229680,cottered brook,stop,2024-05-06 06:30:00
2,barkway,catm.3093,tl38783476,538780,234760,river quin,stop,2024-05-06 06:15:00
3,willingale,cssc.1456,tl58950707,558950,207070,tributary of river roding,start,2024-05-06 06:00:00
4,willingale,cssc.1456,tl58950707,558950,207070,tributary of river roding,stop,2024-05-06 05:30:00
